In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter


from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from scipy.io import savemat

In [2]:
from os import listdir

In [6]:
pickles = ["../pickles/" + x for x in listdir("../pickles") if x[-1]=='p']

In [2]:
p = "pickles/kibera_sub.p"

In [8]:
polution_date_all = []
X = []
Y = []
features = ['d0', 'd1', 'd2', 'd3', 'd4',
       'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14',
       'd15', 'temp','humidity','dir_(degrees)', 'spd_(m/s)']

In [ ]:
def make_df(p):
    alliance = pd.read_pickle(p)
    alliance['d'] = pd.to_datetime(alliance['d'])
    pm25 = alliance.groupby(alliance['dt'].dt.date)['pm25'].mean()
    pm10 = alliance.groupby(alliance['dt'].dt.date)['pm10'].mean()
    polution_dates = set(pm25[pm25>25].index.values).union(set(pm10[pm10>100].index.values))
    polution_dates = set([pd.to_datetime(x).date() for x in polution_dates])
    polution_date_all.append(polution_dates)
    print(len(polution_dates))
    grouped = alliance.groupby(alliance['dt'].dt.date)
    
    
    for name, group in grouped:
        group['hour_x'] = group['dt'].dt.hour
        group = group.drop_duplicates(['hour_x'])

        group = group.set_index('hour_x')

        if len(group) < 18:
            print pd.to_datetime(name).date()
            print len(group)
            continue

        group = group[features]
        group = group.reindex(range(24))
        group = group.fillna(method='pad')
        group = group.fillna(method='bfill')

        X.append(group.as_matrix())
        if pd.to_datetime(name).date() in polution_dates:
            Y.append(1)
            #print pd.to_datetime(name).date()
        else:
            Y.append(0)
    
    

In [3]:
alliance = pd.read_pickle(p)

In [4]:
alliance['d'] = pd.to_datetime(alliance['d'])

In [5]:
alliance.columns.values

array(['Unnamed: 0', 'd', 't', 'date_x', 'no2', 'o3', 'no', 'so2', 'temp',
       'humidity', 'pm1', 'pm25', 'pm10', 'bin0', 'bin1', 'bin2', 'bin3',
       'bin4', 'bin5', 'bin6', 'bin7', 'bin8', 'bin9', 'bin10', 'bin11',
       'bin12', 'bin13', 'bin14', 'bin15', 'd0', 'd1', 'd2', 'd3', 'd4',
       'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14',
       'd15', 'dt', 'date_y', 'nsec', 'altm_(hpa)', 'temp (degree_c)',
       'dew (degree_c)', 'rh_(percentage)', 'dir_(degrees)', 'spd_(m/s)',
       'vis (km)', 'clouds (====)'], dtype=object)

In [23]:
pm25 = alliance.groupby(alliance['dt'].dt.date)['pm25'].mean()

In [7]:
len(pm25), len(pm25[pm25>25])

(249, 73)

In [22]:
pm10 = alliance.groupby(alliance['dt'].dt.date)['pm10'].mean()

In [9]:
len(pm10), len(pm10[pm10>100])

(249, 8)

In [10]:
pm25[pm25>25]

d
2016-01-06    26.787619
2016-01-07    25.248095
2016-01-09    27.730000
2016-01-10    26.187500
2016-02-06    26.582414
2016-02-08    27.608000
2016-02-09    45.528000
2016-03-09    44.701702
2016-04-06    38.355000
2016-05-07    27.114706
2016-05-08    30.022000
2016-05-23    34.773636
2016-05-24    30.735000
2016-05-25    28.948929
2016-05-27    50.521053
2016-05-28    30.304828
2016-05-31    35.993333
2016-06-06    35.198571
2016-06-07    63.118889
2016-06-08    39.922500
2016-06-09    26.238966
2016-06-13    31.312400
2016-06-15    42.070938
2016-06-17    37.602222
2016-06-18    38.268500
2016-06-20    28.439104
2016-06-21    25.995185
2016-06-22    26.751739
2016-06-23    26.657143
2016-06-24    49.473750
                ...    
2016-07-27    45.645000
2016-07-29    27.698621
2016-07-30    28.902105
2016-08-06    25.926538
2016-08-13    35.804516
2016-08-16    29.172258
2016-08-17    25.946400
2016-08-18    56.073125
2016-08-21    25.850741
2016-08-26    30.140833
2016-08-27    

In [11]:
pm10[pm10>100]

d
2016-02-08    114.842000
2016-06-15    129.880000
2016-07-06    121.768696
2016-07-23    117.365000
2016-07-24    156.142500
2016-07-27    107.644167
2016-08-18    119.978125
2016-08-29    103.630571
Name: pm10, dtype: float64

In [12]:
both = alliance.groupby(['d'])['pm10','pm25'].mean()

In [25]:
polution_dates = set(pm25[pm25>25].index.values).union(set(pm10[pm10>100].index.values))
polution_dates = set([pd.to_datetime(x).date() for x in polution_dates])

In [26]:
polution_dates

{datetime.date(2016, 5, 9),
 datetime.date(2016, 5, 11),
 datetime.date(2016, 5, 23),
 datetime.date(2016, 5, 24),
 datetime.date(2016, 5, 25),
 datetime.date(2016, 5, 27),
 datetime.date(2016, 5, 28),
 datetime.date(2016, 5, 31),
 datetime.date(2016, 6, 1),
 datetime.date(2016, 6, 2),
 datetime.date(2016, 6, 4),
 datetime.date(2016, 6, 6),
 datetime.date(2016, 6, 7),
 datetime.date(2016, 6, 8),
 datetime.date(2016, 6, 11),
 datetime.date(2016, 6, 13),
 datetime.date(2016, 6, 15),
 datetime.date(2016, 6, 17),
 datetime.date(2016, 6, 18),
 datetime.date(2016, 6, 20),
 datetime.date(2016, 6, 21),
 datetime.date(2016, 6, 22),
 datetime.date(2016, 6, 23),
 datetime.date(2016, 6, 24),
 datetime.date(2016, 6, 25),
 datetime.date(2016, 6, 27),
 datetime.date(2016, 6, 28),
 datetime.date(2016, 7, 1),
 datetime.date(2016, 7, 5),
 datetime.date(2016, 7, 6),
 datetime.date(2016, 7, 7),
 datetime.date(2016, 7, 8),
 datetime.date(2016, 7, 9),
 datetime.date(2016, 7, 12),
 datetime.date(2016, 7, 13)

In [27]:
len(polution_dates)

75

In [15]:
grouped = alliance.groupby(alliance['dt'].dt.date)

In [16]:
features = ['d0', 'd1', 'd2', 'd3', 'd4',
       'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14',
       'd15', 'temp','humidity','dir_(degrees)', 'spd_(m/s)']

In [37]:
X = []
Y = []

In [49]:
X = []
Y = []
counter = 0
for name, group in grouped:
    group['hour_x'] = group['dt'].dt.hour
    group = group.drop_duplicates(['hour_x'])
   
    group = group.set_index('hour_x')

    if len(group) < 18:
        print pd.to_datetime(name).date()
        print len(group)
        continue

    group = group[features]
    group = group.reindex(range(24))
    group = group.fillna(method='pad')
    group = group.fillna(method='bfill')
            
    X.append(group.as_matrix())
    if pd.to_datetime(name).date() in polution_dates:
        Y.append(1)
        #print pd.to_datetime(name).date()
    else:
        Y.append(0)
    

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


2016-05-06
2
2016-05-29
16
2016-05-30
13
2016-06-06
17
2016-06-15
15
2016-06-16
11
2016-06-18
17
2016-06-28
16
2016-07-01
15
2016-07-05
13
2016-07-06
13
2016-07-15
15
2016-07-19
17
2016-07-23
11
2016-07-24
4
2016-07-25
13
2016-07-26
9
2016-07-27
11
2016-07-28
13
2016-08-01
14
2016-08-02
9
2016-08-03
15
2016-08-10
17
2016-08-11
17
2016-08-18
11
2016-08-20
13
2016-08-21
17
2016-08-25
17
2016-08-26
11
2016-09-11
16
2016-09-12
15
2016-09-13
16
2016-09-16
15
2016-09-20
16
2016-09-21
16
2016-09-23
14
2016-10-01
15
2016-10-03
15
2016-10-11
16
2016-10-12
15
2016-10-13
12
2016-10-14
15
2016-10-15
17
2016-10-16
11
2016-10-17
12
2016-10-19
8
2016-10-30
17
2016-11-23
17
2016-12-05
17
2016-12-06
11
2016-12-15
13
2016-12-19
11
2016-12-20
12
2016-12-24
17
2016-12-27
11
2016-12-28
15
2017-01-03
13
2017-01-04
7
2017-01-05
17
2017-01-06
15
2017-01-07
14
2017-01-08
14
2017-01-09
11
2017-01-10
5


In [50]:
sum(Y)

58

In [41]:
features = ['d0', 'd1', 'd2', 'd3', 'd4',
       'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14',
       'd15', 'temp','humidity','dir_(degrees)', 'spd_(m/s)']


def make_x_y(pickle):
    pos = 0
    neg = 0
    alliance = pd.read_pickle(pickle)
    alliance['d'] = pd.to_datetime(alliance['d'])
    pm25 = alliance.groupby(alliance['dt'].dt.date)['pm25'].mean()
    pm10 = alliance.groupby(alliance['dt'].dt.date)['pm10'].mean()
    polution_dates = set(pm25[pm25>20].index.values).union(set(pm10[pm10>100].index.values))
    polution_dates = set([pd.to_datetime(x).date() for x in polution_dates])
    
    grouped = alliance.groupby(alliance['dt'].dt.date)
    
    for name, group in grouped:
        group['hour_x'] = group['dt'].dt.hour
        group = group.drop_duplicates(['hour_x'])

        group = group.set_index('hour_x')

        if len(group) < 18:
            #print pd.to_datetime(name).date()
            #print len(group)
            continue

        group = group[features]
        group = group.reindex(range(24))
        group = group.fillna(method='pad')
        group = group.fillna(method='bfill')

        X.append(group.as_matrix())
        if pd.to_datetime(name).date() in polution_dates:
            Y.append(1)
            pos += 1
            #print pd.to_datetime(name).date()
        else:
            Y.append(0)
            neg += 1
            
    
    print(pos,neg)
    print "finished %s" %pickle
    

In [42]:
#pickles = ["kibera_sub.p", "scholastica_sub.p","unep_sub.p","alliance_sub.p",  "allsaints_sub.p"]
X = []
Y = []
pickles = ["kibera_sub.p", "scholastica_sub.p","alliance_sub.p"]
for p in pickles:
    p = "../pickles/"+p
    make_x_y(p)

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(100, 85)
finished ../pickles/kibera_sub.p
(43, 140)
finished ../pickles/scholastica_sub.p
(40, 124)
finished ../pickles/alliance_sub.p


In [50]:
neg_index = []
pos_index = []

for i,y in enumerate(Y):
    if y==0:
        neg_index.append(i)
    else:
        pos_index.append(i)

In [44]:
len(pos_index)

183

In [45]:
len(neg_index)

349

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [53]:
X_train[:2]

[array([[  301.4216  ,   313.0854  ,   342.8929  ,   193.0784  ,
            77.62543 ,   110.0768  ,    32.27848 ,     0.      ,
             0.      ,     8.776291,     0.      ,     0.      ,
             0.      ,     0.      ,     0.      ,     0.      ,
            19.43    ,    69.8     ,    50.      ,     3.      ],
        [  301.4216  ,   313.0854  ,   342.8929  ,   193.0784  ,
            77.62543 ,   110.0768  ,    32.27848 ,     0.      ,
             0.      ,     8.776291,     0.      ,     0.      ,
             0.      ,     0.      ,     0.      ,     0.      ,
            19.43    ,    69.8     ,    50.      ,     3.      ],
        [  334.1848  ,   313.0854  ,   454.1015  ,   236.9599  ,
           166.3402  ,    84.6745  ,    38.73418 ,    16.00785 ,
            10.31885 ,     8.776291,     0.      ,     0.      ,
             0.      ,     0.      ,     0.      ,     0.      ,
            18.69    ,    72.28    ,    50.      ,     4.      ],
        [  334.1848  ,

In [54]:
import pickle

In [55]:
dic = {'X_train':X_train,"X_test":X_test, "y_train":y_train, "y_test":y_test }
pickle.dump(dic,open("lstm_xy_new.p",'w'))